# Bidirectional GRU

In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable

## Initialize Input Sequence Randomly

In [2]:
random_input = Variable(torch.FloatTensor(5, 1, 1).normal_(), requires_grad=False)
random_input[:, 0, 0]

tensor([ 0.2487,  0.7082, -0.0741, -1.3125, -0.7955])

In [3]:
reverse_random_input = random_input[np.arange(4, -1, -1), :, :]
reverse_random_input

tensor([[[-0.7955]],

        [[-1.3125]],

        [[-0.0741]],

        [[ 0.7082]],

        [[ 0.2487]]])

## Initialize a GRU Layer

In [4]:
gru_layer = torch.nn.GRU(input_size=1, hidden_size=1, num_layers=1, batch_first=False, bidirectional=False)
gru_layer

GRU(1, 1)

In [5]:
gru_layer.weight_hh_l0.data

tensor([[ 0.3004],
        [-0.3121],
        [ 0.0787]])

## Initialize a Bidirectional GRU Layer

<img src='images/gru-bidirectional-architecture.png' width=50% />

In [6]:
bi_gru_layer = torch.nn.GRU(input_size=1, hidden_size=1, num_layers=1, batch_first=False, bidirectional=True)
bi_gru_layer

GRU(1, 1, bidirectional=True)

In [7]:
bi_gru_layer.weight_hh_l0.data

tensor([[-0.8681],
        [ 0.0275],
        [ 0.2440]])

## Match Weight of Both Layers

In [18]:
gru_layer.weight_ih_l0 = bi_gru_layer.weight_ih_l0_reverse
gru_layer.weight_hh_l0 = bi_gru_layer.weight_hh_l0_reverse
gru_layer.bias_ih_l0 = bi_gru_layer.bias_ih_l0_reverse
gru_layer.bias_hh_l0 = bi_gru_layer.bias_hh_l0_reverse

## Feed Input Sequence into Both Networks

In [24]:
output, hidden = gru_layer(reverse_random_input)

In [25]:
bi_output, bi_hidden = bi_gru_layer(random_input)

## Check The Output

In [27]:
output[:, 0, 0]

tensor([ 0.4581,  0.7898,  0.4958,  0.0642, -0.0585], grad_fn=<SelectBackward>)

In [28]:
bi_output[:, 0, :]

tensor([[ 0.1079, -0.0585],
        [ 0.0461,  0.0642],
        [ 0.1999,  0.4958],
        [ 0.4606,  0.7898],
        [ 0.5263,  0.4581]], grad_fn=<SelectBackward>)

## Check The Hidden

In [29]:
hidden

tensor([[[-0.0585]]], grad_fn=<ViewBackward>)

In [30]:
bi_hidden[:]

tensor([[[ 0.5263]],

        [[-0.0585]]], grad_fn=<SliceBackward>)

---

## Simple Example

In [33]:
forward_output = torch.tensor(range(5))
backward_output = torch.tensor(list(reversed(range(5))))
print('Forward Output:')
print(forward_input)
print('Backward Output:')
print(backward_input)

Forward Output:
tensor([0, 1, 2, 3, 4])
Backward Output:
tensor([4, 3, 2, 1, 0])


In [34]:
hidden = torch.cat((forward_output, backward_output), dim=0).view(1, 1, -1)
print('Hidden Output:')
print(hidden, hidden.shape)

Hidden Output:
tensor([[[0, 1, 2, 3, 4, 4, 3, 2, 1, 0]]]) torch.Size([1, 1, 10])


In [35]:
hidden_reshaped = hidden.view(1, 1, 2, -1)
print('Reshaped Hidden:')
print(hidden_reshaped, hidden_reshaped.shape)

Reshaped Hidden:
tensor([[[[0, 1, 2, 3, 4],
          [4, 3, 2, 1, 0]]]]) torch.Size([1, 1, 2, 5])


---